Testing the tactile gym

In [1]:
from tactile_gym.rl_envs.demo_rl_env_base import demo_rl_env
from tactile_gym.rl_envs.exploration.edge_follow.edge_follow_env import EdgeFollowEnv

pybullet build time: Jan 20 2023 16:26:58


First we set up some variables of what we want within the envirnment. 

Number of iterations is how long our training loop will be

max_steps is how many steps each iteration can ake within the environment

show_gui allows us to view or not view a gui showing whats going on

This is the same with show tactle

Rendering is required for showing the gui - it is quicker without



In [2]:

seed = int(0)
num_iter = 100
max_steps = 250
show_gui = True
show_tactile = True
render = True
print_info = False
image_size = [128, 128]


Set up the environment

In [3]:
env_modes = {
    # which dofs can have movement
    "movement_mode": "xy",

    # specify arm
    "arm_type": "ur5",

    # specify tactile sensor
    "tactile_sensor_name": "tactip",
    # "tactile_sensor_name": "digit",
    # "tactile_sensor_name": "digitac",

    # the type of control used
    # "control_mode": "TCP_position_control",
    'control_mode': 'TCP_velocity_control',

    # add variation to embed distance to optimise for
    # warning, don't use rand height when controlling z unless
    # including embed distance in observation
    # 'noise_mode':'fixed_height',
    "noise_mode": "rand_height",

    # which observation type to return
    'observation_mode': 'oracle',
    # "observation_mode": "tactile",
    # 'observation_mode':'visual',
    # 'observation_mode':'visuotactile',

    # which reward type to use (currently only dense)
    "reward_mode": "dense"
    # 'reward_mode':'sparse'
}
env = EdgeFollowEnv(
    max_steps=max_steps,
    env_modes=env_modes,
    show_gui=show_gui,
    show_tactile=show_tactile,
    image_size=image_size
)

# set seed for deterministic results
env.seed(seed)
env.action_space.np_random.seed(seed)


argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org


Activate GUI is selected

In [4]:
# create controllable parameters on GUI
action_ids = []
min_action = env.min_action
max_action = env.max_action
if show_gui:

    if env_modes["movement_mode"] == "xy":
        action_ids.append(env._pb.addUserDebugParameter("dX", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dY", min_action, max_action, 0))

    elif env_modes["movement_mode"] == "xyz":
        action_ids.append(env._pb.addUserDebugParameter("dX", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dY", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dZ", min_action, max_action, 0))

    elif env_modes["movement_mode"] == "xyRz":
        action_ids.append(env._pb.addUserDebugParameter("dX", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dY", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dRz", min_action, max_action, 0))

    elif env_modes["movement_mode"] == "xyzRz":
        action_ids.append(env._pb.addUserDebugParameter("dX", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dY", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dZ", min_action, max_action, 0))
        action_ids.append(env._pb.addUserDebugParameter("dRz", min_action, max_action, 0))

# run the control loop
#demo_rl_env(env, num_iter, action_ids, show_gui, show_tactile, render, print_info)

In [5]:
# collection loop
for i in range(num_iter):
    r_sum = 0
    o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0
    step = 0

    while not d:

        if show_gui:
            a = []
            for action_id in action_ids:
                a.append(env._pb.readUserDebugParameter(action_id))
        else:
            a = env.action_space.sample()

        # step the environment
        o, r, d, info = env.step(a)

        if print_info:
            print("")
            print("Step: ", step)
            print("Act:  ", a)
            print("Obs:  ")
            for key, value in o.items():
                if value is None:
                    print("  ", key, ":", value)
                else:
                    print("  ", key, ":", value.shape)
            print("Rew:  ", r)
            print("Done: ", d)
        # render visual + tactile observation
        if render:
            render_img = env.render()

        r_sum += r
        step += 1

        q_key = ord("q")
        r_key = ord("r")
        keys = env._pb.getKeyboardEvents()
        if q_key in keys and keys[q_key] & env._pb.KEY_WAS_TRIGGERED:
            exit()
        elif r_key in keys and keys[r_key] & env._pb.KEY_WAS_TRIGGERED:
            d = True

    print("Total Reward: ", r_sum)

Total Reward:  -43.75193943579056
Total Reward:  -43.752371808237626
Total Reward:  -43.75191886884572
Total Reward:  -43.759206173897205
Total Reward:  -43.74977846050147
Total Reward:  -43.75073885920083
Total Reward:  -43.75706046836393
Total Reward:  -43.75177311475714
Total Reward:  -43.75836015026896
Total Reward:  -43.75127830115847
Total Reward:  -43.7501868941185
Total Reward:  -43.76027131257435
Total Reward:  -43.7516513259499
Total Reward:  -43.75254139180789
Total Reward:  -43.761771379095656
Total Reward:  -43.7544636385033
Total Reward:  -43.758304640054675
Total Reward:  -43.75660863241039
Total Reward:  -43.75305294734375
Total Reward:  -43.761797928391616
Total Reward:  -43.75515286873353
Total Reward:  -43.75357542150951
Total Reward:  -43.75320094554295
Total Reward:  -43.75048256419961
Total Reward:  -43.75194146199684
Total Reward:  -43.75264700208222
Total Reward:  -43.752768846287545
Total Reward:  -43.754573341859164
Total Reward:  -43.75276890965002
Total Rewa

: 

: 